# Chapter 4: The Cross-Entropy Method

A first encounter with an actual RL algorithm. As in, a better one (we hope) than the random agent we built in Chapter 2... :)

In [1]:
import numpy as np
import torch
import torch.nn as nn
from collections import namedtuple

HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [2]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

Note that the net has linear output, even though we want to use the output as a set of probabilities. We _could_ add a softmax layer, but for improved numerical stability we'll instead use `CrossEntropyLoss()` in a minute, which requires raw numeric input and then does softmax & cross-entropy in the same step.

In [3]:
Episode = namedtuple("Episode", field_names=["reward", "steps"])
EpisodeStep = namedtuple("EpisodeStep", field_names=["observation", "action"])


In [4]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.SoftMax(dim=1) # To get a "probability" from the raw output

    while True:
        obs_v = torch.FloatTensor([obs]) # Convert current observation to PyTorch tensor and pass as "batch" i.e. in a list
        act_probs_v = sm(net(obs_v)) # Pass through NN and obtain probabilities
        act_probs = act_probs_v.data.numpy()[0] # Convert bach to numpy array and take first batch element to get 1d array of probs

        action = np.random.choice(len(act_probs), p=act_probs) # Sample an action using those probabilities
        next_obs, reward, is_done, _ = env.step(action)

        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action) # Saving the observation *used to choose the action*
        episode_steps.append(step)

        if is_done:
            # Add episode to store
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            # Prepare for next episode
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()

            if len(batch) == batch_size:
                yield batch # Quit the while loop and return the batch
                batch = []
        
        # Prepare for next iteration
        obs = next_obs
            
        

In [5]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards)) # Used for monitoring only

    # Keep only observations and actions where reward was above specified percentile
    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)

    return train_obs_v, train_act_v, reward_bound, reward_mean # Final two just for tensorboard monitoring